In [61]:
import os
import copy
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import random_split, DataLoader
from sklearn.metrics import classification_report

In [62]:
# Configs
data_dir = '../Preprocessing/New Dataset'
batch_size = 32

start_epeoch=100
end_epoch=501
step=5
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_save_path = 'best_model.pth'
num_classes = 3  # dry, normal, oily

In [63]:
# Transforms
mean = [0.485, 0.456, 0.406]  # ImageNet mean
std = [0.229, 0.224, 0.225]   # ImageNet std

In [64]:
# With augmentation
data_transforms_aug = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}

In [65]:
# Without augmentation
data_transforms_plain = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}

In [66]:
def load_datasets(transforms_dict):
    full_dataset = datasets.ImageFolder(data_dir, transform=transforms_dict['train'])
    val_dataset = datasets.ImageFolder(data_dir, transform=transforms_dict['val'])

    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size

    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader

In [67]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, end_epoch, mode_name):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(start_epeoch,end_epoch,step):
        print(f'\n[{mode_name}] Epoch {epoch + 1}/{end_epoch}')
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Save best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), model_save_path)
                print("✅ Saved Best Model")

        scheduler.step()
        
    print(f'\nBest Validation Acc: {best_acc:.4f}')
    model.load_state_dict(best_model_wts)
    return model

In [68]:
def test_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=['dry', 'normal', 'oily']))

In [69]:
# Function to run entire training
def run_pipeline(use_aug=True):
    print("\n" + "=" * 60)
    print("Using Augmentation" if use_aug else "Without Augmentation")
    print("=" * 60)

    transforms_dict = data_transforms_aug if use_aug else data_transforms_plain
    train_loader, val_loader = load_datasets(transforms_dict)
    dataloaders = {'train': train_loader, 'val': val_loader}

    # Load pretrained ResNet18
    model_ft = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model_ft.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    model_ft = train_model(model_ft, dataloaders, criterion, optimizer, scheduler, end_epoch,
                           mode_name="Aug" if use_aug else "Plain")

    # Test model
    print("\nTesting on Test Set")
    _, test_loader = load_datasets(transforms_dict)
    test_model(model_ft, test_loader)

In [70]:
run_pipeline(use_aug=False)


Without Augmentation

[Plain] Epoch 101/501
Train Loss: 0.7479 Acc: 0.7012
Val Loss: 0.6958 Acc: 0.7048
✅ Saved Best Model

[Plain] Epoch 106/501
Train Loss: 0.5452 Acc: 0.7696
Val Loss: 0.6391 Acc: 0.7238
✅ Saved Best Model

[Plain] Epoch 111/501
Train Loss: 0.4918 Acc: 0.8024
Val Loss: 0.6468 Acc: 0.7048

[Plain] Epoch 116/501
Train Loss: 0.4555 Acc: 0.8089
Val Loss: 0.5693 Acc: 0.7786
✅ Saved Best Model

[Plain] Epoch 121/501
Train Loss: 0.4276 Acc: 0.8351
Val Loss: 0.7196 Acc: 0.7476

[Plain] Epoch 126/501
Train Loss: 0.3472 Acc: 0.8607
Val Loss: 0.5073 Acc: 0.8048
✅ Saved Best Model

[Plain] Epoch 131/501
Train Loss: 0.2628 Acc: 0.9024
Val Loss: 0.5291 Acc: 0.7857

[Plain] Epoch 136/501
Train Loss: 0.2120 Acc: 0.9202
Val Loss: 0.4994 Acc: 0.8095
✅ Saved Best Model

[Plain] Epoch 141/501
Train Loss: 0.1714 Acc: 0.9375
Val Loss: 0.5239 Acc: 0.8024

[Plain] Epoch 146/501
Train Loss: 0.1494 Acc: 0.9470
Val Loss: 0.4988 Acc: 0.8167
✅ Saved Best Model

[Plain] Epoch 151/501
Train Loss: